<a href="https://colab.research.google.com/github/DavidP0011/etl/blob/main/etl_mkt_DM_mapeo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title gspread_to_gbq()
def gspread_to_gbq(params: dict) -> None:
    """
    Replica todas o algunas hojas de un Google Sheet en un dataset de BigQuery.

    - Si hay columnas sin cabecera, estas no se exportan.
    - Se reemplazan las celdas vacías ('') por None, permitiendo a BigQuery inferir el tipo.
    - No se incluyen filas cuya primera columna esté vacía o None.
    - El número de filas que se muestra corresponde a las filas finales que se subirán a BigQuery,
      no las originales de la hoja.
    - La key sheet_names puede ser una lista de inclusión o exclusión:
        * Si todos los elementos de sheet_names empiezan con '!', se interpretará como una lista de hojas a excluir.
        * Caso contrario, se interpretará como una lista de hojas a incluir.

    Ejemplos:
        - sheet_names = ["Hoja1", "Hoja2"] -> Solo se replican "Hoja1" y "Hoja2".
        - sheet_names = ["!Hoja3", "!Hoja4"] -> Se replican todas excepto "Hoja3" y "Hoja4".
        - sheet_names = None o no definida -> Se replican todas las hojas.
    """

    # Validación de parámetros
    sheet_id = params.get('sheet_id')
    project_id = params.get('project_id')
    dataset_id = params.get('dataset_id')
    sheet_names = params.get('sheet_names')  # Puede ser None, una lista positiva o negativa

    if not sheet_id:
        raise ValueError("El parámetro 'sheet_id' es obligatorio.")
    if not project_id:
        raise ValueError("El parámetro 'project_id' es obligatorio.")
    if not dataset_id:
        raise ValueError("El parámetro 'dataset_id' es obligatorio.")

    try:
        # Importar y autenticar Google Colab y BigQuery
        from google.cloud import bigquery
        from google.colab import auth
        import pandas as pd
        import gspread
        from google.auth import default
        import re

        # Autenticación de usuario
        auth.authenticate_user()

        # Autenticación de Gspread
        creds, _ = default()
        gc = gspread.authorize(creds)

        # Autenticación de cliente de BigQuery
        bq_client = bigquery.Client(credentials=creds, project=project_id)

        # Determinar si sheet_names es una lista de inclusión o exclusión
        exclude_list = False
        if sheet_names and all(str(name).startswith('!') for name in sheet_names):
            exclude_list = True
            sheet_names = [name[1:] for name in sheet_names]  # Remover el '!' del inicio

        # Abrir el Google Sheet por su ID
        sheet = gc.open_by_key(sheet_id)
        worksheets = sheet.worksheets()

        for ws in worksheets:
            ws_title = ws.title
            # Normalizar el nombre de la hoja para el nombre de la tabla
            sheet_name = re.sub(r'[^A-Za-z0-9_]', '_', ws_title)

            # Filtrar según sheet_names
            if sheet_names:
                if exclude_list and ws_title in sheet_names:
                    print(f"Omitiendo la hoja: {ws_title} (en lista de exclusión)")
                    continue
                if not exclude_list and ws_title not in sheet_names:
                    print(f"Omitiendo la hoja: {ws_title} (no está en la lista de inclusión)")
                    continue

            print(f"Procesando la hoja: {ws_title}")

            # Obtener los datos de la hoja
            data = ws.get_all_records()

            if not data:
                print(f"La hoja '{ws_title}' está vacía. Se omitirá.")
                continue

            # Convertir la hoja a DataFrame
            df = pd.DataFrame(data)

            # Eliminar columnas sin cabecera
            valid_columns = [col for col in df.columns if col and str(col).strip() != '']
            df = df[valid_columns]

            # Reemplazar las cadenas vacías por None
            df = df.replace('', None)

            # Filtrar filas cuya primera columna esté vacía o None
            if not df.empty:
                first_col = df.columns[0]
                df = df[df[first_col].apply(lambda x: x is not None and str(x).strip() != '')]

            if df.empty:
                print(f"Todas las filas de la hoja '{ws_title}' fueron filtradas o no tienen datos en la primera columna. Se omitirá.")
                continue

            # Mostrar información final de la hoja antes de subir
            print("\n"*3)
            print(f"La hoja '{ws_title}' tiene {df.shape[0]} filas y {df.shape[1]} columnas (filtrado y limpieza final).")
            print("Nombres de los campos finales a exportar:")
            print(df.columns.tolist())
            print("Primeras 5 filas de la hoja final:")
            display(df.head())

            # Subir el DataFrame a BigQuery con inferencia automática de tipos
            table_id = f"{project_id}.{dataset_id}.{sheet_name}"

            job_config = bigquery.LoadJobConfig(write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE)
            job = bq_client.load_table_from_dataframe(df, table_id, job_config=job_config)
            job.result()  # Esperar a que se complete el trabajo

            print(f"Datos subidos exitosamente a la tabla '{table_id}'\n")

    except Exception as e:
        print(f"Error general durante la replicación: {e}")


In [ ]:
# @title DM - 01 NOMENCLATURAS Y PRECIOS - PUBLICADO Compartido AD

# Autenticación del usuario en Google Colab
from google.colab import auth
auth.authenticate_user()

# Definir los parámetros necesarios para la replicación de las hojas
params = {
    'sheet_id': '1dn6g6DFXMS8xIXFQAwm1PV4j8TNrXJM1Q-ZV5AyBC8A',  # Reemplaza con el ID de tu Google Sheet
    'project_id': 'animum-dev-datawarehouse',  # Reemplaza con el ID de tu proyecto de GCP
    'dataset_id': 'tablas_mapeo',  # Reemplaza con el nombre de tu dataset en BigQuery
    # Lista opcional de hojas a replicar, puede omitirse para incluir todas las hojas. Si se deben excluir ("!") las hojas Hoja2 y Hoja4
    'sheet_names': ["!NVScriptsProperties", "!DO NOT DELETE - AutoCrat Job Settings"]
}

# Llamar a la función para replicar las hojas de Google Sheets a BigQuery
gspread_to_gbq(params)



Procesando la hoja: PF DEF




La hoja 'PF DEF' tiene 37 filas y 17 columnas (filtrado y limpieza final).
Nombres de los campos finales a exportar:
['Estado', 'Nombre programa formativo - Extendido', 'Nombre programa formativo', 'Versión', 'Debut año', 'Debut conv', 'Último año', 'Último conv', 'Nombre programa formativo código', 'Nª módulos académico', 'Nª módulos TPP', 'Nª módulos TFP', 'Nª módulos totales', 'Método', 'Área', 'Disciplina', 'Categoría']
Primeras 5 filas de la hoja final:


,Estado,Nombre programa formativo - Extendido,Nombre programa formativo,Versión,Debut año,Debut conv,Último año,Último conv,Nombre programa formativo código,Nª módulos académico,Nª módulos TPP,Nª módulos TFP,Nª módulos totales,Método,Área,Disciplina,Categoría
0,Vigente,Carrera Concept Art de Escenarios y Personajes...,Carrera Concept Art de Escenarios y Personajes,v01,23/24,T1,None,None,A Art EscPer,11,2,1,14,HFX,Arte y diseño,Concept art,Carrera
1,Vigente,Carrera Concept Art y Modelado de Escenarios 3...,Carrera Concept Art y Modelado de Escenarios 3D,v01,23/24,T1,None,None,A ArtMod Esc,12,1,1,14,HFX,Arte y diseño,Concept art,Carrera
2,Vigente,Carrera Concept Art y Modelado de Personajes 3...,Carrera Concept Art y Modelado de Personajes 3D,v01,22/23,T3,None,None,A ArtMod Per,14,2,1,17,HFX,Arte y diseño,Concept art,Carrera
3,Vigente,Carrera Vfx y Producción 3D con Houdini y Auto...,Carrera Vfx y Producción 3D,v01,22/23,T3,None,None,A VFX-P3D,13,1,1,15,HFX,Postproducción,Vfx,Carrera
4,Vigente,Carrera Arte 3D con Autodesk Suite para Cine y...,Carrera Arte 3D,v02,23/24,T3,None,None,A Art,12,2,1,15,HFX,Producción 3D,Producción 3D,Carrera


/usr/local/lib/python3.10/dist-packages/google/cloud/bigquery/_pandas_helpers.py:570: UserWarning: Pyarrow could not determine the type of columns: Último año, Último conv.
  warnings.warn(


Datos subidos exitosamente a la tabla 'animum-dev-datawarehouse.tablas_mapeo.PF_DEF'

Procesando la hoja: MOD DEF




La hoja 'MOD DEF' tiene 93 filas y 15 columnas (filtrado y limpieza final).
Nombres de los campos finales a exportar:
['Estado', 'Área', 'Disciplina', 'Naming módulo tipo', 'Método', 'Nombre módulo', 'Nombre módulo código', 'Módulo tarifa', 'Duración meses', 'Nº clases preparación', 'Nº clases lectivas', 'Nº clases cierre', 'Nº clases totales', 'T1 T3', 'T2 T4']
Primeras 5 filas de la hoja final:


,Estado,Área,Disciplina,Naming módulo tipo,Método,Nombre módulo,Nombre módulo código,Módulo tarifa,Duración meses,Nº clases preparación,Nº clases lectivas,Nº clases cierre,Nº clases totales,T1 T3,T2 T4
0,Vigente,Arte y diseño,Dibujo,Aca,HFJ,Dibujo Digital y Animación 2D - Aca 1,M Dib Anm 2D Ac1,Máster Dib Anm 2D,3,1,10,1,12,TRUE,FALSE
1,Vigente,Arte y diseño,Dibujo,Aca,HFJ,Fundamentos de la Animación 2D - Aca 1,Fnd Anm Per 2D Ac1,Fundamentos 2D,3,1,10,1,12,TRUE,FALSE
2,Vigente,Arte y diseño,Dibujo,Aca,HFJ,Fundamentos del Dibujo - Aca 1,Fnd Dib Ac1,Fundamentos 2D,3,1,10,1,12,TRUE,FALSE
3,Vigente,Arte y diseño,Concept art,Aca,HFX,Concept Art de Escenarios - Aca 1,CArt Esc Ac1,Avanzado,3,1,10,1,12,TRUE,FALSE
4,Vigente,Arte y diseño,Concept art,Aca,HFX,Concept Art de Escenarios - Aca 2,CArt Esc Ac2,Avanzado,3,1,10,1,12,FALSE,TRUE


...

Datos subidos exitosamente a la tabla 'animum-dev-datawarehouse.tablas_mapeo.PRECIOS_22_06_TD'

Procesando la hoja: PRECIOS 23 06 TD




La hoja 'PRECIOS 23 06 TD' tiene 36 filas y 9 columnas (filtrado y limpieza final).
Nombres de los campos finales a exportar:
['Nombre programa formativo', 'Versión', 'Duración meses', 'Duración años', 'Precio base', 'Descuento', 'Precio', 'Fecha entrada vigor', 'Merma']
Primeras 5 filas de la hoja final:


,Nombre programa formativo,Versión,Duración meses,Duración años,Precio base,Descuento,Precio,Fecha entrada vigor,Merma
0,Curso Iniciación a Maya,v01,10,1,0 €,0 €,0 €,1/06/2023,0%
1,Curso Fundamentos de la Animación 2D,v01,30,3,350 €,0 €,350 €,1/06/2023,0%
2,Curso Fundamentos del Dibujo,v01,30,3,350 €,0 €,350 €,1/06/2023,0%
3,Curso Iniciación a la Producción 3D,v01,30,3,500 €,0 €,500 €,1/06/2023,0%
4,Curso Avanzado Infografía 3D,v01,30,3,2.900 €,0 €,2.900 €,1/06/2023,0%


Datos subidos exitosamente a la tabla 'animum-dev-datawarehouse.tablas_mapeo.PRECIOS_23_06_TD'

Procesando la hoja: PRECIOS 24 06 TD




La hoja 'PRECIOS 24 06 TD' tiene 32 filas y 9 columnas (filtrado y limpieza final).
Nombres de los campos finales a exportar:
['Nombre programa formativo', 'Versión', 'Duración meses', 'Duración años', 'Precio base', 'Descuento', 'Precio', 'Fecha entrada vigor', 'Merma']
Primeras 5 filas de la hoja final:


,Nombre programa formativo,Versión,Duración meses,Duración años,Precio base,Descuento,Precio,Fecha entrada vigor,Merma
0,Curso Iniciación a Maya,v01,10,1,0 €,0 €,0 €,1/06/2024,10%
1,Curso Fundamentos de la Animación 2D,v01,30,3,350 €,0 €,350 €,1/06/2024,10%
2,Curso Fundamentos del Dibujo,v01,30,3,350 €,0 €,350 €,1/06/2024,10%
3,Curso Iniciación a la Producción 3D,v01,30,3,500 €,0 €,500 €,1/06/2024,10%
4,Workshop Avanzado EYR – Animación Personajes,v01,53,4,850 €,0 €,850 €,1/06/2024,0%


Datos subidos exitosamente a la tabla 'animum-dev-datawarehouse.tablas_mapeo.PRECIOS_24_06_TD'

Omitiendo la hoja: DO NOT DELETE - AutoCrat Job Settings (en lista de exclusión)


In [ ]:
# @title CAPTACION_BUDGET

# Autenticación del usuario en Google Colab
from google.colab import auth
auth.authenticate_user()

# Definir los parámetros necesarios para la replicación de las hojas
params = {
    'sheet_id': '18hw6BwZF0tqdl7I_pOhV0j5bGWctaw243Kr-xcZ6uWM',  # Reemplaza con el ID de tu Google Sheet
    'project_id': 'animum-dev-datawarehouse',  # Reemplaza con el ID de tu proyecto de GCP
    'dataset_id': 'tablas_mapeo',  # Reemplaza con el nombre de tu dataset en BigQuery
    # Lista opcional de hojas a replicar, puede omitirse para incluir todas las hojas. Si se deben excluir ("!") las hojas Hoja2 y Hoja4
    'sheet_names': ["EXPORTAR"]
}

# Llamar a la función para replicar las hojas de Google Sheets a BigQuery
gspread_to_gbq(params)



Omitiendo la hoja: 23-24 (no está en la lista de inclusión)
Omitiendo la hoja: 24-25 (no está en la lista de inclusión)
Procesando la hoja: EXPORTAR




La hoja 'EXPORTAR' tiene 104 filas y 6 columnas (filtrado y limpieza final).
Nombres de los campos finales a exportar:
['Semana', 'Año académico', 'Día', 'Año', 'Mes', 'Budget']
Primeras 5 filas de la hoja final:


,Semana,Año académico,Día,Año,Mes,Budget
0,1,24-25,28/10/2024,2024,oct,72
1,2,24-25,04/11/2024,2024,nov,72
2,3,24-25,11/11/2024,2024,nov,72
3,4,24-25,18/11/2024,2024,nov,72
4,5,24-25,25/11/2024,2024,nov,72


Datos subidos exitosamente a la tabla 'animum-dev-datawarehouse.tablas_mapeo.EXPORTAR'



In [ ]:
# prompt: cargar librería whisper para transcripción de audios

!pip install git+https://github.com/openai/whisper.git
import whisper

# Load the Whisper model
model = whisper.load_model("large") # Choose the model size: "tiny", "base", "small", "medium", "large"

# Transcribe the audio file
# # Replace 'audio.mp3' with the actual path to your audio file
# result = model.transcribe("/content/audio.mp3")

# # Print or process the transcribed text
# print(result["text"])

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-3w45zsgq
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-3w45zsgq
  Resolved https://github.com/openai/whisper.git to commit 90db0de1896c23cbfaf0c58bc2d30665f709f170
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.3 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803583 sha256=59ba0c6b112e08c27d08eff7ba65686e5d4abf98ad96a38a8327f620052706b2
  Stored in directory: /tmp/pip-ephem-wheel-cache-41pys74r/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


100%|█████████████████████████████████████| 2.88G/2.88G [00:30<00:00, 99.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# prompt: Tengo cargado whisper, queri transcribir un video /content/drive/MyDrive/ESCRITORIO/HOUDINI DRIVE/T035 - PYRO/L050 FX ALTO NIVEL - JP CAMPUZANO/L050 FX ALTO NIVEL - C00 BIENVENIDA.mp4
# El texto está en español

# Transcribe the video file
result = model.transcribe("/content/drive/MyDrive/ESCRITORIO/HOUDINI DRIVE/T035 - PYRO/L050 FX ALTO NIVEL - JP CAMPUZANO/L050 FX ALTO NIVEL - C00 BIENVENIDA.mp4", language='es')

# Print or process the transcribed text
print(result["text"])

/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Bienvenidos a esta nueva lección de generación de explosiones. Y bueno, para comenzar esta semana dejamos a un lado la escena que llevaba trabajando Andreu, donde hemos aprendido a generar unos meteoritos y una explosión aérea, y nos introducimos en esta nueva semana que es un setup limpio de generación de explosiones. La idea de este setup es que veáis como una pieza no que pertenece a este plano, sino que podéis reutilizar en cualquier otro plano. Todas estas capas que vais a aprender de detalle podéis utilizarlas juntas o separadas, dependiendo del efecto que queráis conseguir y dependiendo del plano con el que estéis trabajando. Por eso al final de esta lección tendréis... un vídeo donde retomamos con Andreu la colocación de estas explosiones. Nosotros generaremos nuestras explosiones en el cero del mundo y luego las colocaremos cuando el meteorito impacta contra el suelo, en el tiempo y en el lugar donde el meteorito, cada uno de esos meteoritos impacta. Entonces podemos generar 